<a href="https://colab.research.google.com/github/hymansun/slh_repo/blob/main/paraformer_v4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pydub
!pip install fastapi
# !pip install modelscope
!pip install pyngrok
!pip install uvicorn
# !pip install funasr
!pip install git+https://github.com/openai/whisper.git

# !pip uninstall -y numpy
# !pip install numpy==1.22.0
# !pip uninstall -y pyarrow
# !pip install pyarrow==3.0.0

# https://3b2c-34-80-60-76.ngrok-free.app/asr?url=https://www.xiaoyuzhoufm.com/episode/644b8e9db7b3c21b651f4eff&act=TEXT
# https://22fd-35-240-130-155.ngrok-free.app/asr?url=https://www.xiaoyuzhoufm.com/episode/6480423c6752b5f9de289b49&act=TEXT

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.7/65.7 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 681.2/681.2 kB 7.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-6.0.0-py3-none-any.whl size=19866 sha256=c61ab5f2a2c2ff62562124a3b80779e7cf61375dbec036918776ef1cbb144f37
  Stored in directory: /root/.cache/pip/wheels/5c/42/78/0c3d438d7f5730451a25f7ac6cbf4391759d22a67576ed7c2c
Successfully built pyngrok
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.0 MB/s eta 0:00:00
  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-240qaoqh
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-240qaoqh
  Resolved https://github.com/openai/whisper.git to commit b91c907694f96

In [2]:
!ngrok config add-authtoken 2S0vnfxJ0gGEOuFKszCI2GkITM8_3VugUQjVgDAsyvUiWcrEb

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
# !pip uninstall -y numpy
# !pip install numpy==1.22.0
# !pip uninstall -y librosa
# !pip install librosa==0.8.1

In [ ]:
# !pip install git+https://github.com/m1guelpf/yt-whisper.git
# !pip install whisper
# !pip install git+https://github.com/openai/whisper.git

In [6]:
!pip show whisper


/bin/bash: line 1: pip2: command not found


In [ ]:
!pip show torch

In [4]:
import os

import requests
from bs4 import BeautifulSoup
from pydub import AudioSegment
import requests
import re
import json
import requests
from fastapi import FastAPI
import whisper
import numpy
# from modelscope.pipelines import pipeline
# from modelscope.utils.constant import Tasks

import nest_asyncio
from pyngrok import ngrok
import uvicorn

save_data_service = 'http://139.198.168.88:8888/data'

NOTION_KEY = "secret_yV4xicfxDkCDqkdqGMT78EyppnsLhFDKUxn698mtUL0"
# DATABASE_ID = "468290681c374438b37e30fb916cb25c" # test1
DATABASE_ID = "19dff37dab8d4c7fad2d94ac2e6eb94e" # test1

headers = {'Authorization': f"Bearer {NOTION_KEY}", 'Content-Type': 'application/json', 'Notion-Version': '2022-06-28'}
_section_time = 30



# !pip install pydub
# !pip install fastapi
# !pip install modelscope
# !pip install pyngrok
# !pip install uvicorn
# https://79b4-35-233-188-114.ngrok-free.app/asr?url=https://www.xiaoyuzhoufm.com/episode/644b8e9db7b3c21b651f4eff&act=TEXT 谁提前给燃油车企出殡了？
# https://7c03-35-233-188-114.ngrok-free.app/asr?url=https://www.xiaoyuzhoufm.com/episode/64b0f90d44d572bfab0a74bb&act=TEXT 富士康败走印度，究竟“谁坑了谁”？
# https://ab96-103-116-72-184.ngrok-free.app/asr?url=https://www.xiaoyuzhoufm.com/episode/6480423c6752b5f9de289b49&act=TEXT
#  E13. 先实事求是，再去谈投资 | 对话南添
# !ngrok config add-authtoken 2S0vnfxJ0gGEOuFKszCI2GkITM8_3VugUQjVgDAsyvUiWcrEb


# =======================================

# 获取节目网页中的节目信息
def get_audio_url(program_url):
    response = requests.get(program_url)
    soup = BeautifulSoup(response.text, 'html.parser')

    # 获取剧集名称、出品机构名称
    raw_title = soup.title.string if soup.title else None  # 获取网页标题
    episode_title = None
    podcast_title = None
    if raw_title:
        s_removed = raw_title.replace("| 小宇宙 - 听播客，上小宇宙", "").strip()
        parts = s_removed.split(" - ")
        episode_title = parts[0]
        podcast_title = parts[1] if len(parts) > 1 else None
    description = None

    # 获取音视频网址
    audio_url = None
    for meta in soup.find_all('meta'):
        if 'og:audio' in meta.attrs.get('property', ''):
            audio_url = meta.attrs.get('content', '')
            break
    return audio_url, episode_title, podcast_title


# print(get_audio_url("https://www.xiaoyuzhoufm.com/episode/6480423c6752b5f9de289b49"))


# 下载并把音频转换成wav格式
def download_and_convert(audio_url):
    # 下载音频文件
    ori_file_name = re.search(r"/([^/]+\.\w+)$", audio_url).group(1)
    out_file_name = ori_file_name + '.wav'
    if os.path.exists(out_file_name):
        return out_file_name
    response = requests.get(audio_url)
    with open(ori_file_name, 'wb') as file:
        file.write(response.content)
    # 转换音频文件格式
    audio = AudioSegment.from_file(ori_file_name)
    audio.export(out_file_name, format='wav')
    # 获取音频时间
    audio_out = AudioSegment.from_wav(out_file_name)

    return out_file_name,seconds_to_minutes(audio_out.duration_seconds)


def seconds_to_minutes(input_time):
    minutes, seconds = divmod(input_time, 60)
    return f"{minutes}:{str(seconds).zfill(2)}"

# 调用阿里paraformer模型识别文字
def get_wav_transcript_paraformer(wav_file):
    p = pipeline('auto-speech-recognition',
                 'damo/speech_paraformer-large-vad-punc_asr_nat-zh-cn-16k-common-vocab8404-pytorch')
    ts = p(wav_file)
    return ts

# 调用whisper模型识别文字
def get_wav_transcript_whisper(wav_file):
    options = {}
    options["language"] = "chinese"
    model = whisper.load_model('large-v2', device='cuda')
    result = model.transcribe(wav_file, verbose=False,
                              append_punctuations= "\"'.。,，!！?？:：”)]}、",
                              **options)
    # print(result)
    return result


app = FastAPI()

@app.get("/asr")
def asr(url: str, act: str):
    if act.upper() == "TEXT":
        print(url)
        audio_url, episode_title, podcast_title = get_audio_url(url)
        if audio_url == '':
            return "节目地址获取错误"
        out_file_name,audio_time = download_and_convert(audio_url)
        if out_file_name == '':
            return "文件下载错误"
        ts =  get_wav_transcript_whisper(out_file_name)
        status_file = save_content_file(url, ts)
        print("保存到青云：" + status_file)
        transcript = format_result(ts)
        convert_date = datetime.now().strftime("%Y-%m-%d %H:%M")
        status_notion = save_content_to_notion(episode_title, url, "XIAOYUZHOU", audio_url,podcast_title,
                               audio_time, convert_date, transcript)
        print("保存到nontion：" + status_notion)
        return transcript
    else:
        return {"url": url, "act": act, "result": "other content"}

# 格式化文本成按段输出
def format_result(data):
    last_segment = data["segments"][-1]
    last_segment_end = last_segment["end"]
    # 设置分组个数
    group_num = round(last_segment_end / _section_time,0)
    # 分割并打印序列
    str_ret = ""
    groups = np.array_split(data["segments"], group_num)
    for i, group in enumerate(groups, start=1):
        group_str = "("+ seconds_to_minutes(int(group[0]['start'])) \
                    + "--" + seconds_to_minutes(int(group[-1]['end'])) + ")" + "："
        for segment in group:
            group_str = group_str + (segment['text'] + "，")
        # 找到最后一个逗号的索引
        last_comma_index = group_str.rfind("，")
        # 使用字符串切片和拼接将最后一个逗号替换为句号
        s_new = group_str[:last_comma_index] + "。"
        str_ret = str_ret + s_new + "\r\n\r\n"
    return str_ret

def pretty_print(response):
    print(json.dumps(response.json(), indent=2))

# 保存数据到notion
def save_content_to_notion(episode_title, url, host_name, audio_url,podcast_title,
                               audio_time, convert_date, transcript):
    data = {
        "episode_title": {"title": [{"text": {"content": episode_title}}]},
        "url": {"rich_text": [{"text": {"content": url}}]},
        "host_name": {"rich_text": [{"text": {"content": host_name}}]},
        "audio_url": {"rich_text": [{"text": {"content": audio_url}}]},
        "podcast_title": {"rich_text": [{"text": {"content": podcast_title}}]},
        "audio_time": {"rich_text": [{"text": {"content": audio_time}}]},
        "convert_date": {"rich_text": [{"text": {"content": convert_date}}]}
    }
    create_url = "https://api.notion.com/v1/pages"
    payload = {
        "parent": {"database_id": DATABASE_ID},
        "properties": data,
        "children": [{"object": "block","type": "paragraph",
                      "paragraph": {
                            "rich_text": [{"type": "text","text": {"content": transcript} }]
                      }}]
    }
    res = requests.post(create_url, headers=headers, json=payload)
    # pretty_print(res)
    return res.status_code

# save_content_to_notion("episode_title", "url", "host_name", "audio_url","podcast_title",
                              #  "audio_time", "convert_date", "transcript")


# 保存结构体到青云服务器
def save_content_file(filename, ts):
    data = ts
    # filename = filename.replace("http://", "").replace("https://", "")
    params = {
        'filename': filename
    }
    response = requests.post(save_data_service, params=params, data=json.dumps(data))
    print(response)
    print(response.status_code)
    return response.status_code


# ngrok_tunnel = ngrok.connect(8000)
# print("Public URL:", ngrok_tunnel.public_url)
# nest_asyncio.apply()
# uvicorn.run(app, port=8000)



{
  "object": "page",
  "id": "df55127a-6955-4607-9bd6-05274b3d68a2",
  "created_time": "2023-08-07T13:09:00.000Z",
  "last_edited_time": "2023-08-07T13:09:00.000Z",
  "created_by": {
    "object": "user",
    "id": "faeff1e1-5c91-4fd9-9e34-ac7f599bcc48"
  },
  "last_edited_by": {
    "object": "user",
    "id": "faeff1e1-5c91-4fd9-9e34-ac7f599bcc48"
  },
  "cover": null,
  "icon": null,
  "parent": {
    "type": "database_id",
    "database_id": "19dff37d-ab8d-4c7f-ad2d-94ac2e6eb94e"
  },
  "archived": false,
  "properties": {
    "url": {
      "id": "%3F%3ET%7C",
      "type": "rich_text",
      "rich_text": [
        {
          "type": "text",
          "text": {
            "content": "url",
            "link": null
          },
          "annotations": {
            "bold": false,
            "italic": false,
            "strikethrough": false,
            "underline": false,
            "code": false,
            "color": "default"
          },
          "plain_text": "url",
  

In [4]:
import os
import json

import requests

NOTION_KEY = os.environ.get("NOTION_KEY")
NOTION_KEY = "secret_yV4xicfxDkCDqkdqGMT78EyppnsLhFDKUxn698mtUL0"
DATABASE_ID = "468290681c374438b37e30fb916cb25c"
headers = {'Authorization': f"Bearer {NOTION_KEY}", 'Content-Type': 'application/json', 'Notion-Version': '2022-06-28'}

def pretty_print(response):
    print(json.dumps(response.json(), indent=2))

def create_page_in_db(data: dict,page_content: str):
    create_url = "https://api.notion.com/v1/pages"
    payload = {
        "parent": {"database_id": DATABASE_ID},
        "properties": data,
        "children": [{"object": "block","type": "paragraph",
                      "paragraph": {
                            "rich_text": [{"type": "text","text": {"content": page_content} }]
                      }}]
    }
    res = requests.post(create_url, headers=headers, json=payload)
    pretty_print(res)
    print(res.status_code)
    return res

from datetime import datetime, timezone

title = "Test Title7"
author = "hank"
description = "Test Description7"
# published_date = datetime.now().astimezone(timezone.utc).isoformat()
published_date = datetime.now().strftime("%Y-%m-%d %H:%M")

data = {
    "URL": {"title": [{"text": {"content": description}}]},
    "Title": {"rich_text": [{"text": {"content": title}}]},
    "Author": {"rich_text": [{"text": {"content": author}}]},
    # "Published": {"date": {"start": published_date, "end": None}}
    "Published": {"rich_text": [{"text":{"content": published_date}}]}
}
content = "Hello world"
create_page_in_db(data, content)

{
  "object": "page",
  "id": "bd13fbe8-4a9f-4cc5-bf71-354c8774c596",
  "created_time": "2023-08-07T12:23:00.000Z",
  "last_edited_time": "2023-08-07T12:23:00.000Z",
  "created_by": {
    "object": "user",
    "id": "faeff1e1-5c91-4fd9-9e34-ac7f599bcc48"
  },
  "last_edited_by": {
    "object": "user",
    "id": "faeff1e1-5c91-4fd9-9e34-ac7f599bcc48"
  },
  "cover": null,
  "icon": null,
  "parent": {
    "type": "database_id",
    "database_id": "46829068-1c37-4438-b37e-30fb916cb25c"
  },
  "archived": false,
  "properties": {
    "Title": {
      "id": "%5C%3BWB",
      "type": "rich_text",
      "rich_text": [
        {
          "type": "text",
          "text": {
            "content": "Test Title7",
            "link": null
          },
          "annotations": {
            "bold": false,
            "italic": false,
            "strikethrough": false,
            "underline": false,
            "code": false,
            "color": "default"
          },
          "plain_text": "

<Response [200]>